# Fine-tuning of the Marian model (Estonian to English)

In [ ]:
!pip install evaluate openai-whisper datasets peft transformers==4.45
!pip install sacrebleu unbabel-comet
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 32.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("csv", data_files="/content/drive/My Drive/dataset_asr.csv")

# raw_datasets = raw_datasets.select_columns(["transcription","translation"])

raw_datasets = raw_datasets["train"].train_test_split(test_size=0.83, shuffle=False)

print(raw_datasets)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['transcription', 'sentence', 'translation', 'hypothesis_clean', 'sentence_clean', 'translation_clean'],
        num_rows: 170
    })
    test: Dataset({
        features: ['transcription', 'sentence', 'translation', 'hypothesis_clean', 'sentence_clean', 'translation_clean'],
        num_rows: 830
    })
})


Show the training pairs (transcription-translation)

In [ ]:
raw_datasets["train"][:12]["hypothesis_clean"]

[' ey ey ég vel hvort ey ',
 ' kõik kirjeldatud probleimid on sirlele pähe toonud ka elust loobumise mõtteid ',
 ' lõpliku otsuse peaks viss sportlasa suhtes tegema järgmisel nädalal ',
 ' puhkust on vaja taastamiseks et tööd saaks teha selge peaga ',
 ' rakett või battari on iga inimese enda valik ',
 ' ma ei kohanud õpinguta jooksul ühtegi stereotüübsed nohikmatemaatikud ',
 ' õlletootjate sõnul tuleneb alkoholivaba õllahind kallimast tehnoloogiast ',
 ' nætendid mängiti páril mæku ljubbu päeval ',
 ' tagasiside on ariadne sõnul olnud väga pozitiivne ',
 ' senúa behiteld súr tepsust ',
 ' paastamine ei seis näinult hoidus loobumises ',
 ' kui sa mängid nüksa tugevama vastu siis see on väga oluline omadus ']

In [ ]:
raw_datasets["train"][:12]["translation_clean"]

['no no and again no ',
 'all these problems had made sirli consider ending her life ',
 'the fis should make their final decision regarding the athletes next week ',
 'they needed the vacation to recover so that they could work with a clear head ',
 'a rocket or a battery is every person s personal choice ',
 'i never met a stereotypical nerdy mathematician during my studies ',
 'according to beer manufacturers non alcoholic beer s price is formed by the more expensive technology used ',
 'the play was performed on a couple of days at the end of may ',
 'according to ariadne feedback has been very positive ',
 'this requires great accuracy from the constructor ',
 'fasting is not only about giving up food ',
 'if you play against someone slightly stronger this is a very important quality ']

In [ ]:
raw_datasets["train"][:12]["transcription"]

[' Ey, ey, ég vel hvort ey.',
 ' Kõik kirjeldatud probleimid on Sirlele pähe toonud ka elust loobumise mõtteid.',
 ' Lõpliku otsuse peaks viss sportlasa suhtes tegema järgmisel nädalal.',
 ' Puhkust on vaja taastamiseks, et tööd saaks teha selge peaga.',
 ' Rakett või battari on iga inimese enda valik.',
 ' Ma ei kohanud õpinguta jooksul ühtegi stereotüübsed nohikmatemaatikud.',
 ' Õlletootjate sõnul tuleneb alkoholivaba Õllahind kallimast tehnoloogiast.',
 ' Nætendid mängiti páril mæku ljubbu päeval.',
 ' Tagasiside on Ariadne sõnul olnud väga pozitiivne.',
 ' Senúa behiteld súr tepsust.',
 ' Paastamine ei seis näinult hoidus loobumises.',
 ' Kui sa mängid nüksa tugevama vastu, siis see on väga oluline omadus.']

In [ ]:
raw_datasets["train"][:12]["translation"]

['No, no, and again, no.',
 'All these problems had made Sirli consider ending her life.',
 'The FIS should make their final decision regarding the athletes next week.',
 'They needed the vacation to recover so that they could work with a clear head.',
 'A rocket or a battery is every person’s personal choice.',
 'I never met a stereotypical nerdy mathematician during my studies.',
 'According to beer manufacturers, non-alcoholic beer’s price is formed by the more expensive technology used.',
 'The play was performed on a couple of days at the end of May.',
 'According to Ariadne, feedback has been very positive.',
 'This requires great accuracy from the constructor.',
 'Fasting is not only about giving up food.',
 'If you play against someone slightly stronger, this is a very important quality.']

<p style="page-break-after:always;"></p>

Now we load the pre-trained tokenizer for the NLLB model and apply it to the Estonian-English pair:

In [ ]:
max_tok_length = 275

from transformers import AutoTokenizer

checkpoint = "Helsinki-NLP/opus-mt-et-en"
# from flores200_codes import flores_codes
src_code = "et"
tgt_code = "en"
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    padding=True,
    pad_to_multiple_of=8,
    src_lang=src_code,
    tgt_lang=tgt_code,
    truncation=False,
    max_length=max_tok_length,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/825k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


We can apply the tokenizer function to any dataset taking advantage that Hugging Face Datasets are [Apache Arrow](https://arrow.apache.org) files stored on the disk, so you only keep the samples you ask for loaded in memory.

To keep the data as a dataset, we will use the [Dataset.map() function](https://huggingface.co/docs/datasets/en/package_reference/main_classes#datasets.Dataset.map). This also allows us some extra flexibility, if we need more preprocessing done than just tokenization. The map() method works by applying a function on each element of the dataset.

In our case, each sample pair is going to be preprocessed according to the training needs of the model that is to be finetuned:

In [ ]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["hypothesis_clean"],
        text_target=sample["translation_clean"],
    )
    return model_inputs

<p style="page-break-after:always;"></p>

Now, we can apply the preprocess_function to the raw datasets (training, validation and test):

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

We can take a quick look at the length histogram in the source language:

In [ ]:
dic = {}
for sample in tokenized_datasets["train"]:
    sample_length = len(sample["input_ids"])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1, max_tok_length + 1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

 6   1
 7   1
 8   2
 9   4
10   2
11   5
12   8
13   7
14   9
15   7
16   6
17   7
18   9
19   9
20   8
21   3
22   6
23   7
24   9
25   7
26   9
27   5
28   4
29   3
30   5
31   5
32   4
33   4
34   5
35   3
38   3
40   1
43   1
46   1


Checking a sample after filtering by maximum number of tokens:

bitsandbytes is a quantization library with a Transformers integration. With this integration, you can quantize a model to 8 or 4-bits and enable many other options by configuring the BitsAndBytesConfig class. For example, you can:

<ul>
<li>set load_in_4bit=True to quantize the model to 4-bits when you load it</li>
<li>set bnb_4bit_quant_type="nf4" to use a special 4-bit data type for weights initialized from a normal distribution</li>
<li>set bnb_4bit_use_double_quant=True to use a nested quantization scheme to quantize the already quantized weights</li>
<li>set bnb_4bit_compute_dtype=torch.bfloat16 to use bfloat16 for faster computation</li>
</ul>


In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

Pass the quantization_config to the from_pretrained method.

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/299M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Next, you should call the prepare_model_for_kbit_training() function to preprocess the quantized model for training.

In [ ]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=False,
    gradient_checkpointing_kwargs={"use_reentrant": False},
)

[LoRA (Low-Rank Adaptation of Large Language Models)](https://huggingface.co/docs/peft/task_guides/lora_based_methods) is a [parameter-efficient fine-tuning (PEFT)](https://huggingface.co/docs/peft/index) technique that significantly reduces the number of trainable parameters. It works by inserting a smaller number of new weights into the model and only these are trained. This makes training with LoRA much faster, memory-efficient, and produces smaller model weights (a few hundred MBs), which are easier to store and share.

<p style="page-break-after:always;"></p>

Each PEFT method is defined by a PeftConfig class that stores all the important parameters for building a PeftModel. For example, to train with LoRA, load and create a LoraConfig class and specify the following parameters:

<ul>
<li>task_type: the task to train for (sequence-to-sequence language modeling in this case)</li>
<li>r: the dimension of the low-rank matrices</li>
<li>lora_alpha: the scaling factor for the low-rank matrices</li>
<li>target_modules: determine what set of parameters are adapted</li>
<li>lora_dropout: the dropout probability of the LoRA layers</li>
</ul>

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

Once LoRA and the quantization are setup, create a quantized PeftModel with the get_peft_model() function. It takes a quantized model and the LoraConfig containing the parameters for how to configure a model for training with LoRA.

In [ ]:
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 589,824 || all params: 75,392,000 || trainable%: 0.7823


The function that is responsible for putting together samples inside a batch is called a collate function. It is an argument you can pass when you build a DataLoader, the default being a function that will just convert your samples to PyTorch tensors and concatenate them. This is not possible in our case since the inputs we have are not all of the same size. We have deliberately postponed the padding, to only apply it as necessary on each batch and avoid having over-long inputs with a lot of padding.

To do this in practice, we have to define a collate function that will apply the correct amount of padding to the items of the dataset we want to batch together. Fortunately, the Transformers library provides us with such a function via DataCollatorForSeq2Seq that takes a tokenizer when you instantiate it (to know which padding token to use, and whether the model expects padding to be on the left or on the right of the inputs), so we will also need to instantiate the model first to provide it to the collate function:

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=lora_model, pad_to_multiple_of=8
)

## Evaluation

The last thing to define for our Seq2SeqTrainer is how to compute the metrics to evaluate the predictions of our model with respect to references. To this purpose, we use the [Evaluate library](https://huggingface.co/docs/evaluate) which includes the definition of generic and task-specific metrics. In our case, we use the [BLEU metric](https://huggingface.co/spaces/evaluate-metric/bleu), or to be more precise, [sacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu). You can see a simple example of usage below:

In [ ]:
from evaluate import load

metric = load("sacrebleu")

<p style="page-break-after:always;"></p>

We need to define a function compute_metrics to compute BLEU scores at each epoch. The example below performs a basic post-processing to decode the predictions into texts:

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace negative ids in the labels as we can't decode them.
    # labels = np.where(labels < 0, labels, tokenizer.pad_token_id)
    for i in range(len(labels)):
        labels[i] = [tokenizer.pad_token_id if j < 0 else j for j in labels[i]]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

<p style="page-break-after:always;"></p>

## Training

The first step before we can define our [Trainer](https://huggingface.co/docs/transformers/en/main_classes/trainer#trainer) is to define a [Seq2SeqTrainingArguments class](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments) that will contain all the hyperparameters the Trainer will use for training and evaluation. The only compulsory argument you have to provide is a directory where the trained model will be saved, as well as the checkpoints along the way. For all the rest, you can set them depending on the recommendations from the model developers:

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
model_name = checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-est-to-eng",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=15,
    predict_with_generate=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Once we have our model, we can define a Trainer by passing it all the objects constructed up to now — the model, the training_args, the training and validation datasets, the tokenizer, the data collator and the compute_metrics function:

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    lora_model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

To fine-tune the model on our dataset, we just have to call the [train() function](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Trainer.train) of our Trainer:

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,2.320202,20.963300,22.096400
2,No log,2.247129,22.485400,21.669900
3,No log,2.201089,22.976800,21.328900
4,No log,2.176477,23.140500,21.481900
5,No log,2.163572,23.376600,21.234900
6,No log,2.152977,23.405000,21.208400
7,No log,2.145933,23.527100,21.562700
8,No log,2.142364,23.787900,21.257800
9,No log,2.139107,23.769000,21.204800
10,No log,2.137776,23.584600,21.316900


TrainOutput(global_step=330, training_loss=2.4268512147845644, metrics={'train_runtime': 1872.5622, 'train_samples_per_second': 1.362, 'train_steps_per_second': 0.176, 'total_flos': 25527346790400.0, 'train_loss': 2.4268512147845644, 'epoch': 15.0})

## Evaluation of the fine tuned NLLL model on the Europarl-ST test set

Let us first load the default inference parameters of NLLB:

In [ ]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(
    checkpoint,
)

print(generation_config)

GenerationConfig {
  "bad_words_ids": [
    [
      58865
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 58865,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 4,
  "pad_token_id": 58865,
  "renormalize_logits": true
}



We load the speech transcriptions (and reference translations) of Europarl-ST test set from the csv file generated in the experiment L4.1. Then, we prepare the test set in batches to be translated:

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("csv", data_files="/content/drive/My Drive/dataset_asr.csv")

print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['transcription', 'sentence', 'translation', 'hypothesis_clean', 'sentence_clean', 'translation_clean'],
        num_rows: 1000
    })
})


Each sample pair is preprocessed according to the training needs of the model that has been finetuned:

In [ ]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["hypothesis_clean"],
        text_target=sample["translation_clean"],
    )
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
test_batch_size = 32
batch_tokenized_test = tokenized_datasets["train"].batch(test_batch_size)

Batching examples:   0%|          | 0/1000 [00:00<?, ? examples/s]

Processing in batches to add padding and converting to tensors, then perform inference with num_beams = 1 and do_sample = False, that is, greedy search.

In [ ]:
number_of_batches = len(batch_tokenized_test["hypothesis_clean"])
output_sequences = []
for i in range(number_of_batches):
    inputs = tokenizer(
        batch_tokenized_test["hypothesis_clean"][i],
        max_length=max_tok_length,
        truncation=False,
        return_tensors="pt",
        padding=True,
    )
    output_batch = lora_model.generate(
        generation_config=generation_config,
        input_ids=inputs["input_ids"].cuda(),
        attention_mask=inputs["attention_mask"].cuda(),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_code),
        max_length=max_tok_length,
        num_beams=1,
        do_sample=False,
    )
    output_sequences.extend(output_batch.cpu())

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2855: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


We can recover the decoded predictions and references by applying [batch_decode](https://huggingface.co/docs/transformers/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.batch_decode) of the tokenizer

In [ ]:
decoded_preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [ ]:
references = tokenizer.batch_decode(
    tokenized_datasets["train"]["labels"], skip_special_tokens=True
)

Let's take a closer look at some decoded predictions and references:

In [ ]:
references[:5]

['no no and again no',
 'all these problems had made sirli consider ending her life',
 'the fis should make their final decision regarding the athletes next week',
 'they needed the vacation to recover so that they could work with a clear head',
 'a rocket or a battery is every person s personal choice']

In [ ]:
decoded_preds[:5]

['eny ey ég vel hvort ey',
 'en all the problems described have brought to Sirle the thoughts of giving up life',
 'en the final decision should be made next week on the sports scene',
 'en there is a need for rest to restore work to work with clear head',
 'en rocket or batteri is the choice of each person']

Predictions and references are normalized using the Whisper basic text standardisation/normalization module

In [ ]:
from whisper.normalizers.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

decoded_preds_clean = [normalizer(text) for text in decoded_preds]
references_clean = [normalizer(text) for text in references]

In [ ]:
from evaluate import load

metric = load("sacrebleu")

For evaluation, we use the [Evaluate library](https://huggingface.co/docs/evaluate) which includes the definition of generic and task-specific metrics. In our case, we use the [BLEU metric](https://huggingface.co/spaces/evaluate-metric/bleu), or to be more precise, [sacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu).

In [ ]:
result = metric.compute(predictions=decoded_preds_clean, references=references_clean)
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 20.1


In [ ]:
from evaluate import load

comet_metric = load("comet")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


Compute COMET figures using the [Evaluate library](https://huggingface.co/docs/evaluate) which includes the definition of generic and task-specific metrics.

In [ ]:
comet_score = comet_metric.compute(
    predictions=decoded_preds_clean,
    references=references_clean,
    sources=raw_datasets["train"]["hypothesis_clean"],
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 68.74 %
